In [ ]:
# create object with parameters
# run simulation, have the results stored in a df, also have a summary report
# visualize the results

In [2]:
import datetime 

In [4]:
crop_params = {
    'N stuff': 25,
    'k': 0.7,
    'LAI by stage':[[0, .5, 1, 1.5, 2],[.003, .009, .006, .004, .001]],
    'Max Increase of Roots per day': .03,
    'Partitioning %s Roots': 3, # step functions for roots, leaves, stem, and storage organs
    'Temp Sum for Emergence': 800,
    'Temp Sum for Anthesis': 300,
    'Temp Sum for Maturity': 600,
    'Planting Date': datetime.date(2020, 2, 1)
}

In [ ]:
soil_params = {
    'Initial N': 200,
    'Initial SM': .3,
    'Water Retention': 3, # step function of SM and pF
    'SM @ WP': .09,
    'SM @ FC': .3,
    'SM @ Sat': .5
}

In [ ]:
weather_data = pd.DataFrame()

In [ ]:
management_params = {
    'Amount N': 20,
    'pF to Maintain': 2.2
}

In [ ]:
class model :
    def __init__(crop, soil, weather, management) :
        # read variables 
        model.k = crop_params['k']

In [ ]:
# Nitrogen - apply a standard amount at first and then apply a set amount to an area when it's Nitrogen starved
# Irrigation - maintain a certain pF